In [7]:
import bs4
from typing import List
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
# ------------------ 0. 配置常量 ------------------
OLLAMA_MODEL = "qwen2.5"
EMBED_MODEL = "bge-m3"
URL = "https://lilianweng.github.io/posts/2023-06-23-agent/"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
THREAD_ID = "def234"

In [9]:
# ------------------ 1. 初始化模型与向量库 ------------------
llm = ChatOllama(model=OLLAMA_MODEL, temperature=0)
embeddings = OllamaEmbeddings(model=EMBED_MODEL)
vector_store = InMemoryVectorStore(embeddings)

In [10]:
# ------------------ 2. 文档抓取与切分 ------------------
def load_and_index(url: str) -> InMemoryVectorStore:
    """
    抓取网页 → 按指定规则切分 → 向量化 → 返回已填充的向量库
    """
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs={
            "parse_only": bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header")
            )
        },
    )
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
    )
    splits = text_splitter.split_documents(docs)
    vector_store.add_documents(splits)
    return vector_store


vector_store = load_and_index(URL)

In [11]:
# ------------------ 3. 检索工具 ------------------
@tool(response_format="content_and_artifact")
def retrieve(query: str) -> tuple[str, list]:
    """
    根据用户问题，从已索引的文章中检索最相关片段。
    返回 (拼接后的文本, 原始文档列表)，便于调试
    """
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        f"来源：{doc.metadata}\n内容：{doc.page_content}" for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [12]:
# ------------------ 4. 构建 ReAct Agent ------------------
memory = MemorySaver()
agent_executor = create_react_agent(llm, [retrieve], checkpointer=memory)

In [13]:
# ------------------ 5. 统一对话入口 ------------------
def chat(questions: List[str], thread_id: str = THREAD_ID) -> None:
    """
    批量向 Agent 提问并流式输出回答
    """
    config = {"configurable": {"thread_id": thread_id}}
    for q in questions:
        for event in agent_executor.stream(
            {"messages": [{"role": "user", "content": q}]},
            stream_mode="values",
            config=config,
        ):
            event["messages"][-1].pretty_print()

In [15]:
questions = [
    "任务分解（Task Decomposition）的标准方法是什么？找到答案后，再帮我查阅该方法的常见扩展或改进方案。"
]
chat(questions)

================================ Human Message =================================

任务分解（Task Decomposition）的标准方法是什么？找到答案后，再帮我查阅该方法的常见扩展或改进方案。
================================== Ai Message ==================================

根据检索到的信息，任务分解（Task Decomposition）的标准方法主要包括以下几种：

1. **通过简单的提示进行LLM（大型语言模型）任务分解**：例如，使用类似于“步骤为XYZ。\n1.”或“What are the subgoals for achieving XYZ?”这样的简单提示来引导LLM。

2. **使用特定任务的指令**：比如写小说时可以给出“编写故事大纲”的指令。

3. **通过人类输入进行任务分解**。

这些方法提供了任务分解的不同策略和视角。接下来，我将查找这些方法的一些常见扩展或改进方案。
Tool Calls:
  retrieve (4a85ffc3-d150-4221-8ffd-e3ce1a42f8d6)
 Call ID: 4a85ffc3-d150-4221-8ffd-e3ce1a42f8d6
  Args:
    query: task decomposition common extensions or improvements
================================= Tool Message =================================
Name: retrieve

来源：{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
内容：Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of t